# Building model to predict the postion of horse based on previous three races

#### The 5 parameters for the last race and the 5 parameters for the race before that and 5 parameters for race previous to that (i.e. last, second_last, third_last) for each horse

### Importing necessary libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

# The Dataset
The dataset we got has data split in two CSVs, namely `races.csv` and `runs.csv`.

## `races.csv`
Each line in `races.csv` contains the information about a specific race event. The information contained is as follows
 1. race_id: Unique ID for the race
 2. date: Date of the race
 3. venue: 2 characters representing which race course the race took place in.  
     `ST = Shatin, HV = Happy Valley`
 4. race_no: The race number in that day's meeting.
 5. config: Race track configuration for that race.
 6. surface: A binary number representing race track surface.  
     `1 = dirt, 0 = turf`
 7. distance: Distance of the race, in meters.
 8. going: track condition
 9. horse_ratings: the range of the horse ratings that may participate in this race.
 10. prize: The winning prize, in HK dollars.
 11. race_class: a number representing the class of the race.
 12. sec_time1: time taken by the leader of the race to reach the end of the end of the 1st sectional point (sec)
 13. sec_time2: time taken by the leader of the race to reach the end of the 2nd sectional point (sec)
 14. sec_time3: time taken by the leader of the race to reach the end of the 3rd sectional point (sec)
 15. sec_time4: time taken by the leader of the race to reach the end of the 4th sectional point, if any (sec)
 16. sec_time5: time taken by the leader of the race to reach the end of the 5th sectional point, if any (sec)
 17. sec_time6: time taken by the leader of the race to reach the end of the fourth sectional point, if any (sec)
 18. sec_time7: time taken by the leader of the race to reach the end of the fourth sectional point, if any (sec)
 19. time1: time taken by the leader of the race in the 1st section only (sec)
 20. time2: time taken by the leader of the race in the 2nd section only (sec)
 21. time3: time taken by the leader of the race in the 3rd section only (sec)
 22. time4: time taken by the leader of the race in the 4th section only, if any (sec)
 23. time5: time taken by the leader of the race in the 5th section only, if any (sec)
 24. time6: time taken by the leader of the race in the 6th section only, if any (sec)
 25. time7: time taken by the leader of the race in the 7th section only, if any (sec)
 26. place_combination1: placing horse no (1st)
 27. place_combination2: placing horse no (2nd)
 28. place_combination3: placing horse no (3rd)
 29. place_combination4: placing horse no (4th)
 30. place_dividend1: placing dividend paid (for place_combination1)
 31. place_dividend2: placing dividend paid (for place_combination2)
 32. place_dividend3: placing dividend paid (for place_combination2)
 33. place_dividend4: placing dividend paid (for place_combination2)
 34. win_combination1: winning horse no
 35. win_dividend1: winning dividend paid (for win_combination1)
 36. win_combination2: joint winning horse no, if any
 37. win_dividend2: winning dividend paid (for win_combination2, if any)

## `runs.csv`
Each line in `runs.csv` contains the information about one horse run in one of the races given in `races.csv`. 
 1. race_id: unique identifier for the race
 2. horse_no: the number assigned to this horse, in the race
 3. horse_id: unique identifier for this horse
 4. result: finishing position of this horse in the race
 5. won: whether horse won (1) or otherwise (0)
 6. lengths_behind: finishing position, as the number of horse lengths behind the winner
 7. horse_age: current age of this horse at the time of the race
 8. horse_country: country of origin of this horse
 9. horse_type: sex of the horse, e.g. 'Gelding', 'Mare', 'Horse', 'Rig', 'Colt', 'Filly'
 10. horse_rating: rating number assigned by HKJC to this horse at the time of the race
 11. horse_gear: string representing the gear carried by the horse in the race. An explanation of the codes used may be found on the HKJC website.
 12. declared_weight: declared weight of the horse and jockey, in lbs
 13. actual_weight: actual weight carried by the horse, in lbs
 14. draw: post position number of the horse in this race
 15. position_sec1: position of this horse (ranking) in section 1 of the race
 16. position_sec2: position of this horse (ranking) in section 2 of the race
 17. position_sec3: position of this horse (ranking) in section 3 of the race
 18. position_sec4: position of this horse (ranking) in section 4 of the race, if any
 19. position_sec5: position of this horse (ranking) in section 5 of the race, if any
 20. position_sec6: position of this horse (ranking) in section 6 of the race, if any
 21. behind_sec1: position of this horse (lengths behind leader) in section 1 of the race
 22. behind_sec2: position of this horse (lengths behind leader) in section 2 of the race
 23. behind_sec3: position of this horse (lengths behind leader) in section 3 of the race
 24. behind_sec4: position of this horse (lengths behind leader) in section 4 of the race, if any
 25. behind_sec5: position of this horse (lengths behind leader) in section 5 of the race, if any
 26. behind_sec6: position of this horse (lengths behind leader) in section 6 of the race, if any
 27. time1: time taken by the horse to pass through the 1st section of the race (sec)
 28. time2: time taken by the horse to pass through the 2nd section of the race (sec)
 29. time3: time taken by the horse to pass through the 3rd section of the race (sec)
 30. time4: time taken by the horse to pass through the 4th section of the race, if any (sec)
 31. time5: time taken by the horse to pass through the 5th section of the race, if any (sec)
 32. time6: time taken by the horse to pass through the 6th section of the race, if any (sec)
 33. finish_time: finishing time of the horse in this race (sec)
 34. win_odds: win odds for this horse at start of race
 35. place_odds: place (finishing in 1st, 2nd or 3rd position) odds for this horse at start of race
 36. trainer_id: unique identifier of the horse's trainer at the time of the race
 37. jockey_id: unique identifier of the jockey riding the horse in this race

In [2]:
runs_df = pd.read_csv("runs.csv")
races_df = pd.read_csv("races.csv")

# Understanding the Dataset
Before we can start doing our Machine Learning business, we first need to start with understanding the data. For that, we will need to find out some statistics.

In [221]:
runs_df.head()

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,time2,time3,time4,time5,time6,finish_time,win_odds,place_odds,trainer_id,jockey_id
0,0,1,3917,10,0.0,8.00,3,AUS,Gelding,60,...,21.59,23.86,24.62,NaN,NaN,83.92,9.7,3.7,118,2
1,0,2,2157,8,0.0,5.75,3,NZ,Gelding,60,...,21.99,23.30,23.70,NaN,NaN,83.56,16.0,4.9,164,57
2,0,3,858,7,0.0,4.75,3,NZ,Gelding,60,...,21.59,23.90,24.22,NaN,NaN,83.40,3.5,1.5,137,18
3,0,4,1853,9,0.0,6.25,3,SAF,Gelding,60,...,21.83,23.70,24.00,NaN,NaN,83.62,39.0,11.0,80,59
4,0,5,2796,6,0.0,3.75,3,GB,Gelding,60,...,21.75,23.22,23.50,NaN,NaN,83.24,50.0,14.0,9,154


In [6]:
runs_df.shape

(79447, 37)

> + runs dataset contains 79447 rows and 37 features.

In [222]:
races_df.head()

,race_id,date,venue,race_no,config,surface,distance,going,horse_ratings,prize,...,place_combination3,place_combination4,place_dividend1,place_dividend2,place_dividend3,place_dividend4,win_combination1,win_dividend1,win_combination2,win_dividend2
0,0,1997-06-02,ST,1,A,0,1400,GOOD TO FIRM,40-15,485000.0,...,6.0,NaN,36.5,25.5,18.0,NaN,8,121.0,NaN,NaN
1,1,1997-06-02,ST,2,A,0,1200,GOOD TO FIRM,40-15,485000.0,...,4.0,NaN,12.5,47.0,33.5,NaN,5,23.5,NaN,NaN
2,2,1997-06-02,ST,3,A,0,1400,GOOD TO FIRM,60-40,625000.0,...,13.0,NaN,23.0,23.0,59.5,NaN,11,70.0,NaN,NaN
3,3,1997-06-02,ST,4,A,0,1200,GOOD TO FIRM,120-95,1750000.0,...,10.0,NaN,14.0,24.5,16.0,NaN,5,52.0,NaN,NaN
4,4,1997-06-02,ST,5,A,0,1600,GOOD TO FIRM,60-40,625000.0,...,1.0,NaN,15.5,28.0,17.5,NaN,2,36.5,NaN,NaN


In [7]:
races_df.shape

(6349, 37)

> + races dataset contains 6349 rows and 37 features.

## Merging above two dataaset based on `race_id` as it will be helpfull for model building process.

In [5]:
horse = pd.merge(runs_df, races_df,on='race_id')

In [229]:
horse.shape

(79447, 73)

#### Our dataset contain one column having race date. We will convert it's datatype to datetime for further analysis. 

In [231]:
horse.date = pd.to_datetime(horse.date)

In [10]:
horse['date'].head()

0   1997-06-02
1   1997-06-02
2   1997-06-02
3   1997-06-02
4   1997-06-02
Name: date, dtype: datetime64[ns]

# As our objective is to predict position of horse, our dataset should have all horses with necessary details. 
> To achieve our goal we will need to find out the list of all horses that participated in race.

In [12]:
unique_horses = horse.horse_id.unique()
unique_horses = list(unique_horses)

+ unique_horses is the list of all horses participated in race.

## Let's create three dataset having details of last race, second last race and third last race for each horse

In [13]:
last_race = pd.DataFrame()
second_last_race = pd.DataFrame()
third_last_race = pd.DataFrame()

# For the Last race
+ created new dataframe named 'last_race' with race details for all horses

In [14]:
for i in unique_horses:
    last_race = last_race.append(horse[horse['horse_id'] == i].iloc[-1:,:])

+ Records for the last race --->

In [15]:
last_race.head()

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,place_combination3,place_combination4,place_dividend1,place_dividend2,place_dividend3,place_dividend4,win_combination1,win_dividend1,win_combination2,win_dividend2
20419,1629,12,3917,12,0.0,14.75,3,AUS,Gelding,60,...,9.0,NaN,41.5,21.0,17.0,NaN,8,147.5,NaN,NaN
21880,1745,13,2157,8,0.0,11.50,3,NZ,Gelding,60,...,9.0,NaN,33.0,66.5,37.0,NaN,1,90.5,NaN,NaN
14961,1191,1,858,9,0.0,12.50,3,NZ,Gelding,60,...,6.0,NaN,17.0,25.0,35.5,NaN,7,40.5,NaN,NaN
3709,295,7,1853,14,0.0,12.25,3,SAF,Gelding,60,...,11.0,NaN,35.5,75.5,44.5,NaN,10,99.0,NaN,NaN
9895,791,8,2796,3,0.0,2.25,3,GB,Gelding,60,...,8.0,NaN,16.0,22.5,14.0,NaN,9,49.0,NaN,NaN


# For the second Last race
+ created new dataframe named 'second_last_race' with race details for all horses

In [16]:
for i in unique_horses:
    second_last_race = second_last_race.append(horse[horse['horse_id'] == i].iloc[-2:-1,:])

+ Records for the second last race --->

In [17]:
second_last_race.head()

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,place_combination3,place_combination4,place_dividend1,place_dividend2,place_dividend3,place_dividend4,win_combination1,win_dividend1,win_combination2,win_dividend2
19168,1530,14,3917,7,0.0,6.25,3,AUS,Gelding,60,...,8.0,NaN,16.0,19.5,53.0,NaN,4,30.5,NaN,NaN
21550,1719,12,2157,9,0.0,5.75,3,NZ,Gelding,60,...,2.0,NaN,22.0,43.0,14.0,NaN,6,69.5,NaN,NaN
13708,1091,14,858,11,0.0,5.25,3,NZ,Gelding,60,...,3.0,NaN,107.0,26.0,11.5,NaN,6,574.5,NaN,NaN
3170,253,6,1853,11,0.0,5.50,3,SAF,Gelding,60,...,9.0,NaN,13.5,32.0,27.5,NaN,12,33.0,NaN,NaN
9200,735,8,2796,2,0.0,0.25,3,GB,Gelding,60,...,6.0,NaN,13.5,33.5,14.5,NaN,11,27.5,NaN,NaN


# For the third Last race
+ created new dataframe named 'third_last_race' with race details for all horses

In [18]:
for i in unique_horses:
    third_last_race = third_last_race.append(horse[horse['horse_id'] == i].iloc[-3:-2,:])

+ Records for the third last race --->

In [19]:
third_last_race.head()

,race_id,horse_no,horse_id,result,won,lengths_behind,horse_age,horse_country,horse_type,horse_rating,...,place_combination3,place_combination4,place_dividend1,place_dividend2,place_dividend3,place_dividend4,win_combination1,win_dividend1,win_combination2,win_dividend2
18728,1492,14,3917,8,0.0,7.50,3,AUS,Gelding,60,...,7.0,NaN,17.5,34.0,46.0,NaN,1,48.0,NaN,NaN
20414,1629,7,2157,11,0.0,14.50,3,NZ,Gelding,60,...,9.0,NaN,41.5,21.0,17.0,NaN,8,147.5,NaN,NaN
12543,999,14,858,4,0.0,2.50,3,NZ,Gelding,60,...,3.0,NaN,12.0,47.5,26.0,NaN,5,20.5,NaN,NaN
2140,171,6,1853,6,0.0,1.75,3,SAF,Gelding,60,...,13.0,NaN,39.0,21.5,58.0,NaN,1,97.5,NaN,NaN
8318,667,9,2796,2,0.0,0.50,3,GB,Gelding,60,...,11.0,NaN,15.5,80.0,39.0,NaN,8,58.0,NaN,NaN


In [2]:
last_race.shape

NameError: name 'last_race' is not defined

In [ ]:
second_last_race.shape

*From the above observation we can say that there 382 horse who have participated in only one race.*

In [20]:
third_last_race.shape

(3823, 73)

# Now we have three different datasets i.e. 
> `1. Dataset for last race details 2. Dataset for Second last race details 3. Dataset for third last details.`

+ We will murge this three dataset to compare the results and better analysis.

### Instead of taking all the features (columns) we will select only columns which are good for our analysis. 

+ Adding required parameter for last race for each horse

In [21]:
final_df = pd.DataFrame()

In [22]:
final_df[['race_id','horse_id','horse_age','horse_type','declared_weight','actual_weight','date','finish_position','class','condition','length_behind','finish_time','trainer_id','jockey_id']] = last_race[[\
          'race_id','horse_id','horse_age','horse_type','declared_weight','actual_weight','date','result','race_class','config','lengths_behind','finish_time','trainer_id','jockey_id']]

In [23]:
final_df.head()

,race_id,horse_id,horse_age,horse_type,declared_weight,actual_weight,date,finish_position,class,condition,length_behind,finish_time,trainer_id,jockey_id
20419,1629,3917,3,Gelding,1011.0,111,1999-07-28,12,5,B,14.75,140.21,132,151
21880,1745,2157,3,Gelding,1033.0,111,1999-09-13,8,5,A+3,11.50,111.83,80,63
14961,1191,858,3,Gelding,1079.0,133,1998-12-09,9,5,A,12.50,72.04,138,2
3709,295,1853,3,Gelding,1112.0,121,1997-09-27,14,5,A,12.25,112.29,80,59
9895,791,2796,3,Gelding,966.0,121,1998-06-24,3,5,A,2.25,101.70,9,145


In [24]:
final_df.shape

(4405, 14)

+ Adding required parameter for second last race for each horse

In [32]:
final_df1 = pd.DataFrame()

In [33]:
final_df1[['race_id1','horse_id','date1','finish_position1','class1','condition1','length_behind1','finish_time1']] = second_last_race[\
          ['race_id','horse_id','date','result','race_class','config','lengths_behind','finish_time']]

In [34]:
final_df1.head()

,race_id1,horse_id,date1,finish_position1,class1,condition1,length_behind1,finish_time1
19168,1530,3917,1999-06-13,7,5,B+2,6.25,109.60
21550,1719,2157,1999-09-01,9,5,C,5.75,103.96
13708,1091,858,1998-10-25,11,4,C,5.25,86.39
3170,253,1853,1997-09-14,11,5,A+3,5.50,71.48
9200,735,2796,1998-06-03,2,5,A,0.25,100.84


In [35]:
final_df1.shape

(4023, 8)

+ Adding required parameter for third last race for each horse

In [36]:
final_df2 = pd.DataFrame()

In [37]:
final_df2[['race_id2','horse_id','date2','finish_position2','class2','condition2','length_behind2','finish_time2']] = third_last_race[\
                        ['race_id','horse_id','date','result','race_class','config','lengths_behind','finish_time']]

In [38]:
final_df2.head()

,race_id2,horse_id,date2,finish_position2,class2,condition2,length_behind2,finish_time2
18728,1492,3917,1999-05-30,8,5,C+3,7.50,97.89
20414,1629,2157,1999-07-28,11,5,B,14.50,140.20
12543,999,858,1998-09-18,4,4,B+2,2.50,70.54
2140,171,1853,1997-08-10,6,5,A,1.75,101.06
8318,667,2796,1998-02-25,2,5,C+3,0.50,101.97


In [39]:
final_df2.shape

(3823, 8)

## As we have required features for last 3 races now, We will merge all three dataset 

- First we will merge last race and second last race dataset, then merge the results dataset with third last race dataset.

1. Merge of last race and second last dataset

In [40]:
new_df = final_df.merge(final_df1,how='left',on='horse_id')

In [41]:
new_df.head()

,race_id,horse_id,horse_age,horse_type,declared_weight,actual_weight,date,finish_position,class,condition,...,finish_time,trainer_id,jockey_id,race_id1,date1,finish_position1,class1,condition1,length_behind1,finish_time1
0,1629,3917,3,Gelding,1011.0,111,1999-07-28,12,5,B,...,140.21,132,151,1530.0,1999-06-13,7.0,5.0,B+2,6.25,109.60
1,1745,2157,3,Gelding,1033.0,111,1999-09-13,8,5,A+3,...,111.83,80,63,1719.0,1999-09-01,9.0,5.0,C,5.75,103.96
2,1191,858,3,Gelding,1079.0,133,1998-12-09,9,5,A,...,72.04,138,2,1091.0,1998-10-25,11.0,4.0,C,5.25,86.39
3,295,1853,3,Gelding,1112.0,121,1997-09-27,14,5,A,...,112.29,80,59,253.0,1997-09-14,11.0,5.0,A+3,5.50,71.48
4,791,2796,3,Gelding,966.0,121,1998-06-24,3,5,A,...,101.70,9,145,735.0,1998-06-03,2.0,5.0,A,0.25,100.84


In [42]:
new_df.shape

(4405, 21)

2. Merge of result dataset from above merge and third last dataset

In [43]:
horse_df = new_df.merge(final_df2,how='left',on='horse_id')

In [44]:
horse_df.head()

,race_id,horse_id,horse_age,horse_type,declared_weight,actual_weight,date,finish_position,class,condition,...,condition1,length_behind1,finish_time1,race_id2,date2,finish_position2,class2,condition2,length_behind2,finish_time2
4400,6345,2607,3,Horse,1108.0,126,2005-08-28,10,11,A,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4401,6346,2605,3,Horse,1070.0,126,2005-08-28,7,11,A,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4402,6346,2606,3,Filly,999.0,122,2005-08-28,9,11,A,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4403,6347,2093,3,Gelding,1125.0,124,2005-08-28,10,2,A,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN
4404,6348,216,3,Gelding,1096.0,130,2005-08-28,1,2,A,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN


In [46]:
horse_df.shape

(4405, 28)

> Thus we have result dataset (horse_df) with 4405 rows and 28 columns which implies dataset for 4405 unique horse with required features.

+ It will be usefull to know the difference of days between two consecutive race. We can get idea for how much time horse rested after previous race.

> We will calculate date difference with the help of race date.

In [47]:
horse_df['day_diff_last_second_race'] = (horse_df['date'] - horse_df['date1']).dt.days

In [48]:
horse_df['day_diff_second_third_race'] = (horse_df['date1'] - horse_df['date2']).dt.days

In [49]:
horse_df[['day_diff_last_second_race','day_diff_second_third_race']].head()

,day_diff_last_second_race,day_diff_second_third_race
0,45.0,14.0
1,12.0,35.0
2,45.0,37.0
3,13.0,35.0
4,21.0,98.0


+ Now we have two more feature with difference between number of days between last and second last race, also between second last and third last race.

# In our merged dataset for last race, second last race and race previous to that has 'NaN' values at some places because there are some horses participated in only one or two race.

* `We will replace those 'NaN' values to '-' for further analysis.`
* `Also we will drop date column for each race as we already calculated day difference between races.`

In [50]:
ndf=horse_df.drop(['date','date1','date2'],axis = 1)

+ Replacing NaN values with '-'.

In [51]:
ndf = ndf.replace(np.nan,'-')

In [52]:
ndf.horse_type.value_counts()

Gelding    4001
Horse       140
Brown       127
Colt         57
Mare         39
Rig          17
Filly        17
Roan          5
Grey          1
-             1
Name: horse_type, dtype: int64

In [53]:
ndf['class'].value_counts()

4     1479
5     1037
3     1019
2      311
11     301
1      140
13      78
12      22
6       15
0        3
Name: class, dtype: int64

## Let's check for horse which participated in current 3 races

+ We will drop horses from our dataset which played only one or two race

In [54]:
ndf = ndf.loc[~ndf.apply(lambda row: (row=='-').any(), axis=1)]

In [55]:
ndf.shape

(3823, 27)

> + Thus 'ndf' is a dataset having records for horses which participated in all last three races.
+ There are 3823 horses participated in all races.

In [56]:
ndf.head()

,race_id,horse_id,horse_age,horse_type,declared_weight,actual_weight,finish_position,class,condition,length_behind,...,length_behind1,finish_time1,race_id2,finish_position2,class2,condition2,length_behind2,finish_time2,day_diff_last_second_race,day_diff_second_third_race
4320,6304,2595,3,Gelding,1125.0,131,2,3,C,1.0,...,6.25,82.76,6249,11,3,C+3,5.25,69.97,7,14
4321,6339,458,3,Gelding,1044.0,124,14,3,A,8.5,...,8.75,71.93,6249,4,3,C+3,1.75,69.44,18,17
4323,6341,1543,3,Gelding,1127.0,117,2,3,A,0.1,...,0.75,81.9,6252,1,4,C+3,0,82.75,14,21
4324,6341,3331,3,Gelding,1096.0,132,1,3,A,0.0,...,1.25,81.95,6254,1,3,C+3,0,82.24,21,14
4343,6341,87,3,Gelding,1007.0,125,14,3,A,8.0,...,15.5,112.19,6274,14,3,A+3,9,96.46,11,18


In [57]:
ndf[(ndf['finish_position'] == 1) | (ndf['finish_position1'] == 1) | (ndf['finish_position2'] == 1)\
   | (ndf['finish_position'] == 2) | (ndf['finish_position1'] == 2) | (ndf['finish_position2'] == 2)\
|(ndf['finish_position'] == 3) | (ndf['finish_position1'] == 3) | (ndf['finish_position2'] == 3)]['horse_age'].value_counts()

3     942
4     178
5     136
6      88
7      27
2       7
8       4
9       2
10      1
Name: horse_age, dtype: int64

In [58]:
ndf[(ndf['finish_position'] == 1) | (ndf['finish_position1'] == 1) | (ndf['finish_position2'] == 1)\
   | (ndf['finish_position'] == 2) | (ndf['finish_position1'] == 2) | (ndf['finish_position2'] == 2)\
|(ndf['finish_position'] == 3) | (ndf['finish_position1'] == 3) | (ndf['finish_position2'] == 3)]['declared_weight'].value_counts().head()

1102.0    15
1154.0    14
1085.0    13
1113.0    13
1160.0    13
Name: declared_weight, dtype: int64

In [59]:
ndf[(ndf['finish_position'] == 1) | (ndf['finish_position1'] == 1) | (ndf['finish_position2'] == 1)\
   | (ndf['finish_position'] == 2) | (ndf['finish_position1'] == 2) | (ndf['finish_position2'] == 2)\
|(ndf['finish_position'] == 3) | (ndf['finish_position1'] == 3) | (ndf['finish_position2'] == 3)]['actual_weight'].value_counts().head()

126    116
133     87
123     80
121     79
120     74
Name: actual_weight, dtype: int64

In [60]:
ndf.horse_type.value_counts()

Gelding    3627
Brown        95
Horse        52
Colt         16
Rig          15
Mare         10
Filly         5
Roan          2
Grey          1
Name: horse_type, dtype: int64

In [61]:
ndf[(ndf['finish_position'] == 1) | (ndf['finish_position1'] == 1) | (ndf['finish_position2'] == 1)]['horse_type'].value_counts()

Gelding    493
Brown       12
Horse        7
Colt         4
Mare         1
Rig          1
Name: horse_type, dtype: int64

- `As we can see horse type 'Gelding' seem to be top performer`

### Also let's check which horse type is performing well to be 1'st to 3'rd position

In [62]:
ndf[(ndf['finish_position'] == 1) | (ndf['finish_position1'] == 1) | (ndf['finish_position2'] == 1)\
   | (ndf['finish_position'] == 2) | (ndf['finish_position1'] == 2) | (ndf['finish_position2'] == 2)\
|(ndf['finish_position'] == 3) | (ndf['finish_position1'] == 3) | (ndf['finish_position2'] == 3)]['horse_type'].value_counts()

Gelding    1312
Brown        34
Horse        20
Colt          8
Mare          6
Rig           4
Filly         1
Name: horse_type, dtype: int64

- Horse type ' Gelding' is in top performer if we looked for position 1'st to 3'rd.

In [63]:
ndf.trainer_id.value_counts()

97     209
47     199
137    197
55     184
138    179
29     177
118    176
75     175
69     175
80     167
60     166
7      162
117    162
164    161
54     152
4      152
38     142
98     137
128    122
27     112
132    103
73      95
111     93
9       87
63      54
109     45
70      23
52       2
134      1
14       1
18       1
165      1
135      1
121      1
166      1
99       1
82       1
91       1
112      1
116      1
39       1
120      1
171      1
Name: trainer_id, dtype: int64

In [64]:
ndf[(ndf['finish_position'] == 1) | (ndf['finish_position1'] == 1) | (ndf['finish_position2'] == 1)\
   | (ndf['finish_position'] == 2) | (ndf['finish_position1'] == 2) | (ndf['finish_position2'] == 2)\
|(ndf['finish_position'] == 3) | (ndf['finish_position1'] == 3) | (ndf['finish_position2'] == 3)]['trainer_id'].value_counts().head()

118    91
97     89
7      84
29     82
69     74
Name: trainer_id, dtype: int64

+ `Trainer ID '118' seems to give better performance as that trainer id participated in race for 176 times and out of which 91 times it's in 1'st to 3'rd position`

In [65]:
ndf.jockey_id.value_counts()

63     203
64     183
50     183
18     176
175    172
2      159
34     155
138    149
76     146
162    132
40     127
95     120
149    117
110    117
150    112
151     97
21      80
24      75
91      71
177     70
100     62
154     60
157     56
39      54
131     48
23      45
82      45
115     45
10      43
132     40
      ... 
59       4
183      4
182      4
134      4
37       3
74       3
79       3
20       2
117      2
71       2
4        2
49       2
7        2
67       2
179      1
172      1
171      1
163      1
5        1
137      1
122      1
127      1
42       1
48       1
53       1
168      1
113      1
43       1
85       1
123      1
Name: jockey_id, Length: 97, dtype: int64

In [66]:
ndf[(ndf['finish_position'] == 1) | (ndf['finish_position1'] == 1) | (ndf['finish_position2'] == 1)\
   | (ndf['finish_position'] == 2) | (ndf['finish_position1'] == 2) | (ndf['finish_position2'] == 2)\
|(ndf['finish_position'] == 3) | (ndf['finish_position1'] == 3) | (ndf['finish_position2'] == 3 )]['jockey_id'].value_counts().head()

64    91
95    91
2     87
18    85
50    63
Name: jockey_id, dtype: int64

+ `Jockey ID '95' seems to give better performance as that jockey id participated in race for 120 times and out of which 91 times it's in 1'st to 3'rd position`
+ `Also Jockey ID '2' seems to give better performance as that jockey id participated in race for 159 times and out of which 87 times it's in 1'st to 3'rd position'` 

## Below are the details for each horse participated in race with same race class

In [67]:
a = ndf[(ndf['class'] == ndf['class1']) & (ndf['class'] == ndf['class2'])][['horse_id','finish_position','finish_position1','finish_position2']]

In [68]:
a.shape

(2818, 4)

In [69]:
a.head()

,horse_id,finish_position,finish_position1,finish_position2
0,3917,12,7,8
1,2157,8,9,11
3,1853,14,11,6
4,2796,3,2,2
6,911,11,10,10


In [70]:
k = []
for i, j in a.iterrows():
    if (j['finish_position1'] > j['finish_position']):
        k.append(j['horse_id'])
len(k)

1042

In [71]:
l = []
for i, j in a.iterrows():
    if (j['finish_position2'] > j['finish_position1'] > j['finish_position']):
        l.append(j['horse_id'])
len(l)

272

# from the above observation we can say that,
- There are `2818` horse which participated all last 3 races with race class.
- Out of which only `272` horses has shown the improvement in final position i.e. they have achieve better final position as compared to previous race
- Also if we compare only for last two race then there are `1042` horses which has imporvement in their final position

In [72]:
print((272/2818)*100)

9.65223562810504


## Below are the details for each horse participated in race with condition.

In [73]:
a = ndf[(ndf['condition'] == ndf['condition1']) & (ndf['condition1'] == ndf['condition2'])][['horse_id','finish_position','finish_position1','finish_position2']]

In [74]:
a.shape

(305, 4)

In [75]:
a.head()

,horse_id,finish_position,finish_position1,finish_position2
12,727,12,9,11
20,1926,2,11,4
60,3301,14,12,6
64,3105,9,13,8
90,3440,8,9,7


In [76]:
k = []
for i, j in a.iterrows():
    if (j['finish_position1'] > j['finish_position']):
        k.append(j['horse_id'])
len(k)

104

In [77]:
l = []
for i, j in a.iterrows():
    if (j['finish_position2'] > j['finish_position1'] > j['finish_position']):
        l.append(j['horse_id'])
len(l)

23

# from the above observation we can say that,
- There are `305` horse which participated all last 3 races with same condition.
- Out of which only `23` horses has shown the improvement in final position i.e. they have achieve better final position as compared to previous race
- Also if we compare only for last two race then there are `104` horses which has imporvement in their final position

In [78]:
print((23/305)*100)

7.540983606557377


From all the above observation we can say that to predict horse to be in 1'st, 2'nd, 3'rd position following parameteres are helpful,
- Horse Type
- Horse Age
- Trainer
- Jockey
- Weight of Horse

In [79]:
ndf['day_diff_last_second_race'].value_counts().head(8)

21.0    463
28.0    286
14.0    285
35.0    150
25.0    146
18.0    140
17.0    121
11.0    109
Name: day_diff_last_second_race, dtype: int64

In [80]:
ndf[(ndf['finish_position'] == 1) | (ndf['finish_position1'] == 1) | (ndf['finish_position2'] == 1)\
   | (ndf['finish_position'] == 2) | (ndf['finish_position1'] == 2) | (ndf['finish_position2'] == 2)\
|(ndf['finish_position'] == 3) | (ndf['finish_position1'] == 3) | (ndf['finish_position2'] == 3)]['day_diff_last_second_race'].value_counts().head(8)

#day_diff_last_second_race	day_diff_second_third_race

21.0    214
28.0    108
14.0     97
35.0     66
18.0     51
25.0     49
42.0     39
22.0     39
Name: day_diff_last_second_race, dtype: int64

+ As we can observe difference between last and second last race is also important parameter for prediction for horse race
+ We can see there is 463 times difference two races is 21 and out of which 214 times horse came in 1'st to 3'rd position

In [81]:
ndf['day_diff_second_third_race'].value_counts().head(8)

21.0    387
14.0    318
28.0    256
35.0    166
17.0    159
18.0    138
25.0    123
24.0    117
Name: day_diff_second_third_race, dtype: int64

In [82]:
ndf[(ndf['finish_position'] == 1) | (ndf['finish_position1'] == 1) | (ndf['finish_position2'] == 1)\
   | (ndf['finish_position'] == 2) | (ndf['finish_position1'] == 2) | (ndf['finish_position2'] == 2)\
|(ndf['finish_position'] == 3) | (ndf['finish_position1'] == 3) | (ndf['finish_position2'] == 3)]['day_diff_second_third_race'].value_counts().head(8)

#day_diff_last_second_race	day_diff_second_third_race

21.0    175
28.0    123
14.0    120
35.0     60
17.0     53
18.0     48
24.0     42
25.0     41
Name: day_diff_second_third_race, dtype: int64

+ As we can observe difference between second last and third last race is also important parameter for prediction for horse race
+ We can see there is 381 times difference two races is 21 and out of which 175 times horse came in 1'st to 3'rd position

In [83]:
ndf.head()

,race_id,horse_id,horse_age,horse_type,declared_weight,actual_weight,finish_position,class,condition,length_behind,...,length_behind1,finish_time1,race_id2,finish_position2,class2,condition2,length_behind2,finish_time2,day_diff_last_second_race,day_diff_second_third_race
0,1629,3917,3,Gelding,1011.0,111,12,5,B,14.75,...,6.25,109.6,1492,8,5,C+3,7.5,97.89,45,14
1,1745,2157,3,Gelding,1033.0,111,8,5,A+3,11.50,...,5.75,103.96,1629,11,5,B,14.5,140.2,12,35
2,1191,858,3,Gelding,1079.0,133,9,5,A,12.50,...,5.25,86.39,999,4,4,B+2,2.5,70.54,45,37
3,295,1853,3,Gelding,1112.0,121,14,5,A,12.25,...,5.5,71.48,171,6,5,A,1.75,101.06,13,35
4,791,2796,3,Gelding,966.0,121,3,5,A,2.25,...,0.25,100.84,667,2,5,C+3,0.5,101.97,21,98


In [84]:
ndf.columns

Index(['race_id', 'horse_id', 'horse_age', 'horse_type', 'declared_weight',
       'actual_weight', 'finish_position', 'class', 'condition',
       'length_behind', 'finish_time', 'trainer_id', 'jockey_id', 'race_id1',
       'finish_position1', 'class1', 'condition1', 'length_behind1',
       'finish_time1', 'race_id2', 'finish_position2', 'class2', 'condition2',
       'length_behind2', 'finish_time2', 'day_diff_last_second_race',
       'day_diff_second_third_race'],
      dtype='object')

# Support Vector Machine

In [96]:
sv_df = ndf.copy()

In [98]:
sv_df.head()

,race_id,horse_id,horse_age,horse_type,declared_weight,actual_weight,finish_position,class,condition,length_behind,...,length_behind1,finish_time1,race_id2,finish_position2,class2,condition2,length_behind2,finish_time2,day_diff_last_second_race,day_diff_second_third_race
0,1629,3917,3,Gelding,1011.0,111,12,5,B,14.75,...,6.25,109.6,1492,8,5,C+3,7.5,97.89,45,14
1,1745,2157,3,Gelding,1033.0,111,8,5,A+3,11.50,...,5.75,103.96,1629,11,5,B,14.5,140.2,12,35
2,1191,858,3,Gelding,1079.0,133,9,5,A,12.50,...,5.25,86.39,999,4,4,B+2,2.5,70.54,45,37
3,295,1853,3,Gelding,1112.0,121,14,5,A,12.25,...,5.5,71.48,171,6,5,A,1.75,101.06,13,35
4,791,2796,3,Gelding,966.0,121,3,5,A,2.25,...,0.25,100.84,667,2,5,C+3,0.5,101.97,21,98


In [191]:
sv_df['finish_position'] = sv_df['finish_position'].apply(lambda x: 5 if x > 4 else x)

In [192]:
sv_df['finish_position1'] = sv_df['finish_position1'].apply(lambda x: 5 if x > 4 else x)

In [193]:
sv_df['finish_position2'] = sv_df['finish_position2'].apply(lambda x: 5 if x > 4 else x)

In [194]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
#sv_df['horse_type'] = le.fit_transform(sv_df['horse_type'])
sv_df['condition'] = le.fit_transform(sv_df['condition'])

In [195]:
sv_df['condition1'] = le.fit_transform(sv_df['condition1'])

In [196]:
sv_df['condition2'] = le.fit_transform(sv_df['condition2'])

In [197]:
X = sv_df.drop(['race_id', 'horse_id', 'horse_age', 'horse_type', 'declared_weight',
       'actual_weight','trainer_id','jockey_id','race_id1',
       'race_id2', 'day_diff_last_second_race','finish_position',
       'day_diff_second_third_race'],axis = 1)

In [198]:
X.columns

Index(['class', 'condition', 'length_behind', 'finish_time',
       'finish_position1', 'class1', 'condition1', 'length_behind1',
       'finish_time1', 'finish_position2', 'class2', 'condition2',
       'length_behind2', 'finish_time2'],
      dtype='object')

In [199]:
y = sv_df['finish_position']

In [200]:
X.head()

,class,condition,length_behind,finish_time,finish_position1,class1,condition1,length_behind1,finish_time1,finish_position2,class2,condition2,length_behind2,finish_time2
0,5,2,14.75,140.21,5.0,5,3,6.25,109.6,5.0,5,5,7.5,97.89
1,5,1,11.50,111.83,5.0,5,4,5.75,103.96,5.0,5,2,14.5,140.2
2,5,0,12.50,72.04,5.0,4,4,5.25,86.39,4.0,4,3,2.5,70.54
3,5,0,12.25,112.29,5.0,5,1,5.5,71.48,5.0,5,0,1.75,101.06
4,5,0,2.25,101.70,2.0,5,0,0.25,100.84,2.0,5,5,0.5,101.97


In [201]:
y.head()

0    5.0
1    5.0
2    4.0
3    5.0
4    2.0
Name: finish_position, dtype: float64

In [202]:
from sklearn import svm
#clf = svm.SVC(gamma=0.025, C=10)    
clf = svm.SVC(probability=True)    
# gamma is a measure of influence of a data point. It is inverse of distance of influence. C is complexity of the model
# lower C value creates simple hyper surface while higher C creates complex surface

In [203]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [204]:
clf.fit(X_train , y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [205]:
clf.score(X_test, y_test)

0.7410636442894507

In [206]:
predicted = clf.predict(X_test)

In [207]:
q =pd.DataFrame(predicted)
q[0].value_counts()

5.0    1110
1.0      16
2.0      13
3.0       8
Name: 0, dtype: int64

In [208]:
y_test.value_counts()

5.0    830
4.0     90
3.0     79
2.0     74
1.0     74
Name: finish_position, dtype: int64

In [209]:
pred_proba = clf.predict_proba(X_test) # predict_proba function call

In [210]:
pd.DataFrame(pred_proba,columns = [1,2,3,4,5])*100

,1,2,3,4,5
0,2.281603,3.501919,7.464863,19.986256,66.765359
1,4.707484,4.786775,5.802828,6.200071,78.502842
2,1.807755,2.297922,3.600337,5.092170,87.201815
3,2.546607,3.288084,4.167363,5.059966,84.937980
4,3.445250,4.458898,5.366993,5.999270,80.729588
5,4.730370,4.804226,5.845119,6.210543,78.409742
6,4.806734,4.855139,5.860342,6.234251,78.243533
7,4.762830,4.911130,6.967742,6.450500,76.907798
8,1.203220,1.767230,2.351872,3.529715,91.147964
9,3.875111,4.199214,5.276687,5.827944,80.821044


In [211]:
test_df = pd.DataFrame()

In [212]:
to_target_list = ['class', 'condition', 'length_behind', 'finish_time',
       'finish_position1', 'class1', 'condition1', 'length_behind1',
       'finish_time1', 'finish_position2', 'class2', 'condition2',
       'length_behind2', 'finish_time2']

In [213]:
from_target_list = ['class', 'condition', 'length_behind', 'finish_time',
       'finish_position1', 'class', 'condition', 'length_behind',
       'finish_time','finish_position1', 'class1', 'condition1', 'length_behind1',
       'finish_time1']

In [214]:
test_df = X[from_target_list]

In [215]:
test_df.columns = to_target_list

In [216]:
X.head()

,class,condition,length_behind,finish_time,finish_position1,class1,condition1,length_behind1,finish_time1,finish_position2,class2,condition2,length_behind2,finish_time2
0,5,2,14.75,140.21,5.0,5,3,6.25,109.6,5.0,5,5,7.5,97.89
1,5,1,11.50,111.83,5.0,5,4,5.75,103.96,5.0,5,2,14.5,140.2
2,5,0,12.50,72.04,5.0,4,4,5.25,86.39,4.0,4,3,2.5,70.54
3,5,0,12.25,112.29,5.0,5,1,5.5,71.48,5.0,5,0,1.75,101.06
4,5,0,2.25,101.70,2.0,5,0,0.25,100.84,2.0,5,5,0.5,101.97


In [217]:
test_df.head()

,class,condition,length_behind,finish_time,finish_position1,class1,condition1,length_behind1,finish_time1,finish_position2,class2,condition2,length_behind2,finish_time2
0,5,2,14.75,140.21,5.0,5,2,14.75,140.21,5.0,5,3,6.25,109.6
1,5,1,11.50,111.83,5.0,5,1,11.50,111.83,5.0,5,4,5.75,103.96
2,5,0,12.50,72.04,5.0,5,0,12.50,72.04,5.0,4,4,5.25,86.39
3,5,0,12.25,112.29,5.0,5,0,12.25,112.29,5.0,5,1,5.5,71.48
4,5,0,2.25,101.70,2.0,5,0,2.25,101.70,2.0,5,0,0.25,100.84


In [218]:
pred_proba = clf.predict_proba(test_df) # predict_proba function call

In [219]:
pd.DataFrame(pred_proba,columns = [1,2,3,4,5])*100

,1,2,3,4,5
0,4.806748,4.855149,5.860350,6.234257,78.243496
1,4.427110,4.586891,5.618410,6.074669,79.292919
2,4.034263,4.303037,5.383613,5.907436,80.371652
3,4.806732,4.855137,5.860340,6.234250,78.243540
4,34.343062,30.340201,26.858145,6.983220,1.475372
5,0.608796,1.068704,2.717563,7.652898,87.952039
6,2.723704,3.300444,4.399519,5.220108,84.356225
7,4.285167,4.484645,5.537206,6.016980,79.676002
8,3.424229,3.826698,4.962339,5.605048,82.181686
9,1.202963,2.061714,3.826996,8.460678,84.447649
